In [1]:
import os
import pickle
import pandas as pd
import statsmodels.api as sm
import statsmodels.discrete.discrete_model as sd
from itertools import combinations
os.chdir("/Users/vnastl/Seafile/My Library/mpi project causal vs noncausal/tableshift")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
# Open a file in binary read mode using a with statement
with open('experiments_vnastl/saved_datafiles/acspubcov_dset.pkl', 'rb') as f:
    # Unpickle the object
    dset = pickle.load(f)


# with open('experiments_vnastl/acspubcov_causal_data.pkl', 'rb') as f:
#     # Unpickle the object
#     dset = pickle.load(f)

In [7]:
dset

TabularDataset(name='acspubcov', preprocessor_config=PreprocessorConfig(categorical_features='one_hot', numeric_features='normalize', domain_labels='label_encode', passthrough_columns=None, dropna='rows', use_extended_names=False, map_targets=False, cast_targets_to_default_type=False, min_frequency=None, max_categories=None, n_bins=5), config=DatasetConfig(cache_dir='tmp', download=True, random_seed=948324), initialize_data=True, splitter=DomainSplitter(val_size=0.1, random_state=264738, id_test_size=0.1, domain_split_varname='DIS', domain_split_ood_values=['1.0'], domain_split_id_values=None, domain_split_gt_thresh=None, drop_domain_split_col=True, ood_val_size=0.1), grouper=Grouper(features_and_values={'RAC1P': [1], 'SEX': [1]}, drop=False, transformer=None), feature_names=['ST_SD', 'MAR_05', 'SCHL_14', 'ST_IL', 'FER_02', 'ST_OR', 'ST_IN', 'ESP_08', 'SCHL_08', 'ST_WI', 'SCHL_12', 'DIVISION_00', 'DREM_02', 'ST_MT', 'MAR_03', 'ST_MN', 'ST_AL', 'SCHL_01', 'ANC_03', 'ST_CO', 'ESP_06', 'E

In [8]:
dset._df

,PINCP,AGEP,ACS_YEAR,ESP_00,ESP_04,ESP_05,ESP_01,ESP_07,ESP_08,ESP_02,...,DIVISION_01,DIVISION_05,DIVISION_03,DIVISION_04,DIVISION_02,DIVISION_00,DIS,SEX,RAC1P,PUBCOV
0,-0.997577,1.553573,-1.385150,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,-0.997577,0.119740,-1.385150,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,-0.997577,-1.314094,-1.385150,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
3,-0.997577,-0.075783,-1.385150,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
4,1.375031,1.032179,-1.385150,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5916560,0.623706,0.771482,1.442855,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
5916561,0.851081,0.771482,1.442855,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
5916562,-0.997577,-1.118571,1.442855,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
5916563,0.099755,1.423224,1.442855,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,1


In [3]:
dset.features

['AGEP',
 'SEX',
 'RAC1P',
 'DIS',
 'DEAR',
 'DEYE',
 'DREM',
 'ANC',
 'NATIVITY',
 'PUBCOV']

In [4]:
X_id_train, y_id_train, _, _ = dset.get_pandas("train")
X_id_test, y_id_test, _, _ = dset.get_pandas("id_test")
X_id_val, y_id_val, _, _ = dset.get_pandas("validation")

In [5]:
X_id = pd.concat([X_id_train, X_id_test, X_id_val])
y_id = pd.concat([y_id_train,y_id_test,y_id_val])

In [6]:
X_ood_test, y_ood_test, _, _ = dset.get_pandas("ood_test")
X_ood_val, y_ood_val, _, _ = dset.get_pandas("ood_validation")

In [7]:
X_ood = pd.concat([X_ood_test,X_ood_val])
y_ood = pd.concat([y_ood_test,y_ood_val])

In [8]:
for col in X_id.columns:
    if len(X_id[col].unique()) == 1:
        X_id.drop(col,inplace=True,axis=1)
relevant_cols = X_id.columns
X_ood = X_ood[relevant_cols]

In [9]:
X_id

,AGEP,ANC_01,ANC_02,ANC_04,ANC_03,ANC_08,NATIVITY_01,NATIVITY_02,SEX,RAC1P
131971,0.771482,0,0,1,0,0,1,0,0,1
1438291,0.054565,1,0,0,0,0,0,1,1,0
4814590,1.358050,1,0,0,0,0,1,0,0,1
2619274,-1.314094,1,0,0,0,0,1,0,1,1
3388077,-0.727526,0,0,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
3945823,-0.466829,0,1,0,0,0,1,0,0,1
267745,0.054565,0,1,0,0,0,1,0,0,1
4054105,-1.053397,1,0,0,0,0,0,1,1,0
325124,-1.053397,0,0,1,0,0,1,0,0,1


In [10]:
import math

def same_sign(a, b):
    return math.copysign(1, a) == math.copysign(1, b)

# Coefficient estimation

## OLS estimation

In [11]:
ols_X_id = X_id.drop([item for item in set(["ANC_01","NATIVITY_01","ESP_00","MIG_01","ESR_01","MAR_01","CIT_01","FER_00","SCHL_01","DIVISION_01"]).intersection(list(X_id.columns))],axis=1)
for col in ols_X_id:
    if col.startswith("ST"):
        ols_X_id.drop(col,inplace=True,axis=1)
ols_X_id = sm.add_constant(ols_X_id)
ols_X_id.reset_index(drop=True, inplace=True)
ols_y_id = y_id.reset_index(drop=True)

In [12]:
model = sm.OLS(ols_y_id,ols_X_id)
results_linear_id = model.fit()
results_linear_id.summary()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 PUBCOV   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     9081.
Date:                Tue, 24 Oct 2023   Prob (F-statistic):               0.00
Time:                        15:49:43   Log-Likelihood:            -2.6870e+06
No. Observations:             5007812   AIC:                         5.374e+06
Df Residuals:                 5007803   BIC:                         5.374e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3086      0.000    700.336      0.000       0.308       0.309
AGEP            0.0031      0.000     16.605      0.000       0.003       0.004
ANC_02         -0.0420      0.000    -88.625      0.000      -0.043      -0.041
ANC_04          0.0070      0.001     13.506      0.000       0.006       0.008
ANC_03         -0.0240      0.001    -17.365      0.000      -0.027      -0.021
ANC_08          0.0202      0.011      1.834      0.067      -0.001       0.042
NATIVITY_02    -0.0486      0.001    -91.146      0.000      -0.050      -0.048
SEX             0.0082      0.000     21.623      0.000       0.007       0.009
RAC1P          -0.1000      0.000   -231.721      0.000      -0.101      -0.099
==============================================================================
Omnibus:                   875382.325   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1419781.900
Skew:                           1.299   Prob(JB):                         0.00
Kurtosis:                       2.773   Cond. No.                         82.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
ols_X_ood = X_ood.drop([item for item in set(["ANC_01","NATIVITY_01","ESP_00","MIG_01","ESR_01","MAR_01","CIT_01","FER_00","SCHL_01","DIVISION_01"]).intersection(list(X_id.columns))],axis=1)
for col in ols_X_ood:
    if col.startswith("ST"):
        ols_X_ood.drop(col,inplace=True,axis=1)
ols_X_ood = sm.add_constant(ols_X_ood)
ols_X_ood.reset_index(drop=True, inplace=True)
ols_y_ood = y_ood.reset_index(drop=True)

In [14]:
model = sm.OLS(ols_y_ood,ols_X_ood)
results_linear_ood = model.fit()
results_linear_ood.summary()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 PUBCOV   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     2807.
Date:                Tue, 24 Oct 2023   Prob (F-statistic):               0.00
Time:                        15:49:44   Log-Likelihood:            -6.1321e+05
No. Observations:              908753   AIC:                         1.226e+06
Df Residuals:                  908744   BIC:                         1.227e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.6426      0.001    530.389      0.000       0.640       0.645
AGEP            0.0623      0.001    119.905      0.000       0.061       0.063
ANC_02         -0.0300      0.001    -23.984      0.000      -0.032      -0.028
ANC_04         -0.0293      0.001    -21.566      0.000      -0.032      -0.027
ANC_03         -0.0201      0.003     -6.019      0.000      -0.027      -0.014
ANC_08          0.0601      0.031      1.929      0.054      -0.001       0.121
NATIVITY_02    -0.1454      0.002    -76.545      0.000      -0.149      -0.142
SEX             0.0359      0.001     35.851      0.000       0.034       0.038
RAC1P          -0.0552      0.001    -48.120      0.000      -0.057      -0.053
==============================================================================
Omnibus:                  4549685.100   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           141954.747
Skew:                          -0.551   Prob(JB):                         0.00
Kurtosis:                       1.408   Cond. No.                         101.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
coefs = pd.DataFrame({"coefs_id":results_linear_id.params, "coefs_ood":results_linear_ood.params,
                      "diff": (results_linear_ood.params- results_linear_id.params), "rel_diff": (results_linear_ood.params- results_linear_id.params)/results_linear_id.params,
                      "same_sign": [same_sign(results_linear_id.params[i],results_linear_ood.params[i]) for i in range(len(results_linear_id.params))]})
coefs.set_index(ols_X_id.columns,inplace=True)
coefs

,coefs_id,coefs_ood,diff,rel_diff,same_sign
const,0.308617,0.642626,0.334009,1.082276,True
AGEP,0.003137,0.062275,0.059138,18.850982,True
ANC_02,-0.042038,-0.029968,0.012070,-0.287119,True
ANC_04,0.006988,-0.029300,-0.036287,-5.192990,False
ANC_03,-0.024015,-0.020078,0.003937,-0.163946,True
ANC_08,0.020201,0.060060,0.039859,1.973065,True
NATIVITY_02,-0.048638,-0.145423,-0.096786,1.989938,True
SEX,0.008167,0.035944,0.027777,3.401071,True
RAC1P,-0.099962,-0.055170,0.044792,-0.448088,True


## Logit

In [16]:
model = sd.Logit(ols_y_id,ols_X_id)
results_id = model.fit()
results_id.summary()

Optimization terminated successfully.
         Current function value: 0.524683
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 PUBCOV   No. Observations:              5007812
Model:                          Logit   Df Residuals:                  5007803
Method:                           MLE   Df Model:                            8
Date:                Tue, 24 Oct 2023   Pseudo R-squ.:                 0.01314
Time:                        15:49:54   Log-Likelihood:            -2.6275e+06
converged:                       True   LL-Null:                   -2.6625e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.7950      0.002   -330.497      0.000      -0.800      -0.790
AGEP            0.0184      0.001     16.691      0.000       0.016       0.021
ANC_02         -0.2576      0.003    -89.106      0.000      -0.263      -0.252
ANC_04          0.0430      0.003     14.553      0.000       0.037       0.049
ANC_03         -0.1399      0.008    -16.713      0.000      -0.156      -0.123
ANC_08          0.1112      0.061      1.815      0.069      -0.009       0.231
NATIVITY_02    -0.2760      0.003    -88.202      0.000      -0.282      -0.270
SEX             0.0468      0.002     21.302      0.000       0.043       0.051
RAC1P          -0.5460      0.002   -227.287      0.000      -0.551      -0.541
===============================================================================
"""

In [17]:
margins_id = results_id.get_margeff(at='mean',dummy=True)
margins_id.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 PUBCOV
Method:                          dydx
At:                              mean
===============================================================================
                 dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
AGEP            0.0032      0.000     16.692      0.000       0.003       0.004
ANC_02         -0.0425      0.000    -92.984      0.000      -0.043      -0.042
ANC_04          0.0074      0.001     14.441      0.000       0.006       0.008
ANC_03         -0.0231      0.001    -17.395      0.000      -0.026      -0.021
ANC_08          0.0197      0.011      1.762      0.078      -0.002       0.042
NATIVITY_02    -0.0449      0.000    -93.401      0.000      -0.046      -0.044
SEX             0.0081      0.000     21.262      0.000       0.007       0.009
RAC1P          -0.0997      0.000   -216.507      0.000      -0.101      -0.099
===============================================================================
"""

In [18]:
model = sd.Logit(ols_y_ood,ols_X_ood)
results_ood = model.fit()
results_ood.summary()

Optimization terminated successfully.
         Current function value: 0.643275
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 PUBCOV   No. Observations:               908753
Model:                          Logit   Df Residuals:                   908744
Method:                           MLE   Df Model:                            8
Date:                Tue, 24 Oct 2023   Pseudo R-squ.:                 0.01840
Time:                        15:49:57   Log-Likelihood:            -5.8458e+05
converged:                       True   LL-Null:                   -5.9554e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.6051      0.005    111.872      0.000       0.594       0.616
AGEP            0.2701      0.002    118.387      0.000       0.266       0.275
ANC_02         -0.1328      0.006    -24.143      0.000      -0.144      -0.122
ANC_04         -0.1298      0.006    -21.710      0.000      -0.142      -0.118
ANC_03         -0.0903      0.015     -6.155      0.000      -0.119      -0.062
ANC_08          0.2833      0.146      1.946      0.052      -0.002       0.569
NATIVITY_02    -0.6239      0.008    -76.468      0.000      -0.640      -0.608
SEX             0.1595      0.004     35.852      0.000       0.151       0.168
RAC1P          -0.2475      0.005    -47.993      0.000      -0.258      -0.237
===============================================================================
"""

In [19]:
margins_ood = results_ood.get_margeff(at='mean',dummy=True)
margins_ood.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 PUBCOV
Method:                          dydx
At:                              mean
===============================================================================
                 dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
AGEP            0.0623      0.001    118.622      0.000       0.061       0.063
ANC_02         -0.0309      0.001    -23.944      0.000      -0.033      -0.028
ANC_04         -0.0303      0.001    -21.494      0.000      -0.033      -0.027
ANC_03         -0.0210      0.003     -6.089      0.000      -0.028      -0.014
ANC_08          0.0624      0.030      2.049      0.040       0.003       0.122
NATIVITY_02    -0.1511      0.002    -74.665      0.000      -0.155      -0.147
SEX             0.0367      0.001     35.909      0.000       0.035       0.039
RAC1P          -0.0561      0.001    -48.916      0.000      -0.058      -0.054
===============================================================================
"""

In [20]:
margins = pd.DataFrame({"margins_id":margins_id.margeff, "margins_ood":margins_ood.margeff,
                        "diff": (margins_ood.margeff- margins_id.margeff), "rel_diff": (margins_ood.margeff- margins_id.margeff)/margins_id.margeff,
                        "same_sign": [same_sign(margins_id.margeff[i],margins_ood.margeff[i]) for i in range(len(margins_id.margeff))]})
margins.set_index(ols_X_id.drop("const",axis=1).columns,inplace=True)
margins

,margins_id,margins_ood,diff,rel_diff,same_sign
AGEP,0.003167,0.062254,0.059087,18.657413,True
ANC_02,-0.042515,-0.030895,0.011619,-0.273303,True
ANC_04,0.007448,-0.030251,-0.037698,-5.061734,False
ANC_03,-0.023105,-0.021047,0.002059,-0.089098,True
ANC_08,0.019679,0.062417,0.042738,2.171733,True
NATIVITY_02,-0.044918,-0.151125,-0.106207,2.364468,True
SEX,0.008058,0.036718,0.028661,3.556863,True
RAC1P,-0.099702,-0.056091,0.043611,-0.437415,True


# Logistic prediction

TODO: implement logistic prediction for id and ood (all vs causal),

# Powerset

TODO: implement powerset and do xgboost on all powersets
TODO: compare train on ood to performance on train on id (causal & all)

In [21]:
def select_powerset(x):
    # Generate powerset of columns
    powerset = []
    for r in range(len(x.columns) + 1):
        combinations_r = list(combinations(x.columns, r))
        for item in combinations_r:
            powerset.append(list(item))
    for item in powerset:
        if len(item) == 0:
            powerset.remove(item)
    return powerset

In [41]:
from tableshift.models.default_hparams import get_default_config
from tableshift.models.utils import get_estimator
from tableshift.models.training import train

model = "xgb"
config = get_default_config(model, dset)
estimator = get_estimator(model, **config)

In [39]:
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()

For sklearn estimator

In [32]:
powerset = select_powerset(ols_X_id.drop([col for col in list(ols_X_id.columns) if col.startswith("ANC")]+["const"],axis=1))
powerset

[['AGEP'],
 ['NATIVITY_02'],
 ['SEX'],
 ['RAC1P'],
 ['AGEP', 'NATIVITY_02'],
 ['AGEP', 'SEX'],
 ['AGEP', 'RAC1P'],
 ['NATIVITY_02', 'SEX'],
 ['NATIVITY_02', 'RAC1P'],
 ['SEX', 'RAC1P'],
 ['AGEP', 'NATIVITY_02', 'SEX'],
 ['AGEP', 'NATIVITY_02', 'RAC1P'],
 ['AGEP', 'SEX', 'RAC1P'],
 ['NATIVITY_02', 'SEX', 'RAC1P'],
 ['AGEP', 'NATIVITY_02', 'SEX', 'RAC1P']]

In [42]:
from sklearn.metrics import accuracy_score
from  statsmodels.stats.proportion import proportion_confint

power_evaluation = []
for collist in powerset:
    X_train = X_id_train[collist]
    estimator.fit(X_train,y_id_train)

    evaluation = {}
    test_split = "id_test"
    X_te = X_id_test[collist]
    y_te = y_id_test
    yhat_te = estimator.predict(X_te)
    acc = accuracy_score(y_true=y_te, y_pred=yhat_te)
    evaluation[test_split] = acc
    nobs = len(y_te)
    count = nobs*acc
    # beta : Clopper-Pearson interval based on Beta distribution
    acc_conf = proportion_confint(count, nobs, alpha=0.05, method='beta')
    evaluation[test_split + "_conf"] = acc_conf
    print(f"training completed! {test_split} accuracy: {acc:.4f}")

    test_split = "ood_test"
    X_te = X_ood_test[collist]
    y_te = y_ood_test
    yhat_te = estimator.predict(X_te)
    acc = accuracy_score(y_true=y_te, y_pred=yhat_te)
    evaluation[test_split] = acc
    nobs = len(y_te)
    count = nobs*acc
    # beta : Clopper-Pearson interval based on Beta distribution
    acc_conf = proportion_confint(count, nobs, alpha=0.05, method='beta')
    evaluation[test_split + "_conf"] = acc_conf
    print(f"training completed! {test_split} accuracy: {acc:.4f}")
    power_evaluation.append([collist,evaluation])

training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637
training completed! id_test accuracy: 0.7762
training completed! ood_test accuracy: 0.3637